In [154]:
import pandas as pd
from datetime import datetime
import re

In [155]:
def parse_logs(logs):
    parsed_data = []
    base_time = None
    
    for log in logs:
        # Pular logs que não contêm métricas (apenas linhas com "RT=")
        if "RT=" not in log:                
            continue
            
        # Extrair timestamp
        timestamp_str = log.split(' - INFO - ')[0].split('| ')[1].strip()
        
        # Converter para datetime object
        current_time = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S,%f")
        
        # Definir base_time como o primeiro timestamp encontrado
        if base_time is None:
            base_time = current_time
        
        # Calcular tempo relativo em segundos
        time_delta = current_time - base_time
        relative_time = time_delta.total_seconds()
        
        # Dividir a linha usando o padrão de separadores
        parts = log.split(' - INFO - ')[1].split(' | ')
        
        data = {'time_relative': relative_time}
        
        for part in parts:
            # Response Time
            if part.startswith('RT='):
                data['response_time'] = float(part.replace('RT=', '').replace('s', '').strip())
            
            # Active Servers
            elif part.startswith('Active servers='):
                data['active_servers'] = int(part.split('=')[1])
            
            # Dimmer
            elif part.startswith('Dimmer='):
                data['dimmer'] = float(part.split('=')[1])
            
            # Arrival Rate
            elif part.startswith('arrival='):
                data['arrival_rate'] = float(part.split('=')[1].split(' ')[0])
            
            # Total Servers
            elif part.startswith('Total servers='):
                data['total_servers'] = int(part.split('=')[1])
            
            # Diagnosis
            elif part.startswith('Diagnosis: '):
                data['diagnosis'] = part.split('Diagnosis: ')[1]
            
            # Action Plan (tratamento especial para JSON)
            elif part.startswith('Action plan: '):
                action_str = part.split('Action plan: ')[1]
                # Encontrar o início e fim do JSON
                start_idx = action_str.find('{')
                end_idx = action_str.rfind('}') + 1
                if start_idx != -1 and end_idx != 0:
                    data['action_plan'] = action_str[start_idx:end_idx]
                else:
                    data['action_plan'] = action_str
            
            # Result
            elif part.startswith('Result: '):
                data['result'] = part.split('Result: ')[1]
        
        parsed_data.append(data)
    
    return pd.DataFrame(parsed_data)

In [156]:
def parse_diagnosis(logs):
    diagnosis = []
    inside_json = False
    json_lines = []

    for log in logs:
        if "Diagnosis:" in log:
            # Caso 1: diagnóstico simples
            if "{" not in log:
                diag = log.split("Diagnosis: ")[1].strip()
                diagnosis.append({"raw": diag})
            # Caso 2: início de JSON
            else:
                inside_json = True
                json_lines = [log.split("Diagnosis: ")[1]]
        elif inside_json:
            json_lines.append(log)
            # Detecta o final do JSON
            if "}" in log:
                inside_json = False
                json_str = "\n".join(json_lines)
                try:
                    # Remove prefixos como "manager |" antes de tentar o parse
                    cleaned = "\n".join([
                        line.split("Diagnosis: ")[-1].split("|")[-1].strip()
                        for line in json_lines
                    ])
                    diag_json = json.loads(cleaned)
                    diagnosis.append(diag_json)
                except Exception as e:
                    diagnosis.append({"raw": cleaned, "error": str(e)})

    diag_final = []

    for item in diagnosis:
        diag_final.append(str(item))

    return pd.DataFrame(diag_final)

In [157]:
def parse_plans(logs):
    plans = []
    for log in logs:
        if "Action plan:" not in log:
            continue
        p = log.split('Action plan: ')[1]
        plans.append(p)

    return pd.DataFrame(plans)

In [158]:
def call_humans(df):
    result_df = df.copy()
    
    mask = (
        result_df['diagnosis'].str.contains('call_human', na=False) & 
        result_df['plans'].str.contains("danger", na=False)
    )
    result_df['human_calls'] = mask
    
    return result_df

In [159]:
records = []

file_path = "0-judge-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
j0 = parse_logs(records)
j0

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.02,3,0.9,47.60,3
1,26.028,3.28,2,0.9,49.05,2
2,56.064,5.57,3,0.9,49.94,3
3,85.651,6.52,3,0.9,50.26,3
4,115.807,6.33,4,0.9,55.13,4
5,145.697,6.29,5,1.0,59.20,5
6,176.825,6.29,5,1.0,57.75,5
7,206.860,6.29,6,1.0,56.41,6
8,235.705,6.29,5,1.0,55.54,5
9,265.741,6.29,6,1.0,92.43,6


In [160]:
records = []

file_path = "1-judge-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
j1 = parse_logs(records)
j1

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.02,3,0.9,44.59,3
1,26.028,3.31,2,0.9,49.30,2
2,56.061,5.60,3,0.9,49.99,3
3,66.978,5.99,3,0.9,50.01,3
4,80.083,6.41,3,0.9,49.93,3
5,88.487,6.55,3,0.9,50.27,3
6,97.602,6.88,4,0.9,50.51,4
7,115.873,6.59,5,0.9,55.29,5
8,145.288,6.28,6,0.9,59.22,6
9,153.812,5.98,6,0.9,58.65,6


In [161]:
records = []

file_path = "2-judge-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
j2 = parse_logs(records)
j2

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.02,3,0.9,42.60,3
1,26.024,3.30,2,0.9,48.92,2
2,56.057,5.20,3,0.9,49.87,3
3,85.203,5.90,4,0.9,50.14,4
4,116.016,4.66,5,0.9,54.98,5
5,146.963,3.94,6,0.9,59.18,6
6,176.997,3.34,7,0.9,57.78,7
7,205.028,3.05,8,0.9,56.52,8
8,235.067,2.70,7,0.9,55.56,7
9,265.102,3.39,8,0.9,91.07,8


In [162]:
records = []

file_path = "3-judge-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
j3 = parse_logs(records)
j3

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.02,3,0.9,42.77,3
1,26.030,3.31,2,0.9,48.95,2
2,56.067,5.54,3,0.9,49.85,3
3,86.249,6.00,4,0.9,50.26,4
4,115.936,4.23,5,0.9,54.97,5
5,155.085,3.60,5,0.9,58.64,5
6,155.091,3.60,6,0.9,58.64,6
7,175.115,3.23,7,0.9,57.89,7
8,205.150,2.91,8,0.9,56.51,8
9,235.186,2.58,7,0.9,55.55,7


In [163]:
records = []

file_path = "4-judge-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
j4 = parse_logs(records)
j4

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.02,3,0.9,42.72,3
1,26.027,3.31,2,0.9,48.95,2
2,56.061,5.54,3,0.9,49.85,3
3,86.391,6.51,4,0.9,50.33,4
4,115.899,5.64,5,0.9,54.98,5
5,146.281,5.03,6,0.9,59.21,6
6,176.681,4.26,7,0.9,57.83,7
7,206.283,3.86,6,0.9,56.46,6
8,236.321,3.42,7,0.9,55.54,7
9,266.355,4.00,8,0.9,92.43,8


In [164]:
records = []

file_path = "5-judge-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
j5 = parse_logs(records)
j5

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.02,3,0.9,43.24,3
1,26.026,3.31,2,0.9,48.95,2
2,56.061,5.54,3,0.9,49.85,3
3,86.532,6.55,4,0.9,50.34,4
4,115.331,5.45,5,0.9,54.76,5
5,145.955,4.85,6,0.9,59.22,6
6,176.887,4.10,6,0.9,57.79,6
7,206.546,3.71,5,0.9,56.47,5
8,236.583,3.29,6,0.9,55.53,6
9,266.620,3.93,7,0.9,92.74,7


In [165]:
records = []

file_path = "baseline-3.txt"
with open(file_path, "r") as f:
    for line in f:
        if "manager" in line:
            records.append(line)
b = parse_logs(records)
b

,time_relative,response_time,active_servers,dimmer,arrival_rate,total_servers
0,0.000,0.03,3,0.9,49.73,3
1,26.028,3.56,2,0.9,50.08,2
2,56.063,5.91,3,0.9,50.70,3
3,86.096,6.93,3,0.9,50.72,3
4,116.130,7.65,3,0.9,54.71,3
5,146.163,8.29,3,0.9,59.05,3
6,176.196,7.93,3,0.9,57.42,3
7,206.230,6.86,3,0.9,56.84,3
8,236.263,6.12,2,0.9,55.99,2
9,266.294,6.22,2,0.9,95.16,2


In [166]:
new_df = pd.DataFrame([{
    'metric': 'mean_rt',
    'baseline':b['response_time'].mean(),
    'j0': j0['response_time'].mean(),
    'j1': j1['response_time'].mean(),
    'j2': j2['response_time'].mean(),
    'j3': j3['response_time'].mean(),
    'j4': j4['response_time'].mean(),
    'j5': j5['response_time'].mean(),
}, {
    'metric': 'mean_active_servers', 
    'baseline': b['active_servers'].mean(),
    'j0': j0['active_servers'].mean(),
    'j1': j1['active_servers'].mean(),
    'j2': j2['active_servers'].mean(),
    'j3': j3['active_servers'].mean(),
    'j4': j4['active_servers'].mean(),
    'j5': j5['active_servers'].mean(),
}, {
    'metric': 'mean_dimmer',
    'baseline': b['dimmer'].mean(),
    'j0': j0['dimmer'].mean(),
    'j1': j1['dimmer'].mean(),
    'j2': j2['dimmer'].mean(),
    'j3': j3['dimmer'].mean(),
    'j4': j4['dimmer'].mean(),
    'j5': j5['dimmer'].mean(),
}, {
    'metric':'max_rt',
    'baseline': b['response_time'].max(),
    'j0': j0['response_time'].max(),
    'j1': j1['response_time'].max(),
    'j2': j2['response_time'].max(),
    'j3': j3['response_time'].max(),
    'j4': j4['response_time'].max(),
    'j5': j5['response_time'].max(),
}, {
    'metric':'min_rt',
    'baseline': b['response_time'].min(),
    'j0': j0['response_time'].min(),
    'j1': j1['response_time'].min(),
    'j2': j2['response_time'].min(),
    'j3': j3['response_time'].min(),
    'j4': j4['response_time'].min(),
    'j5': j5['response_time'].min(),

}

])

new_df_transposed = new_df.set_index('metric').T

new_df_transposed.index.name = 'system'

new_df_transposed

metric,mean_rt,mean_active_servers,mean_dimmer,max_rt,min_rt
system,,,,,
baseline,6.493810,2.333333,0.90000,8.29,0.03
j0,5.826667,5.571429,0.97619,6.52,0.02
j1,4.630769,8.807692,0.90000,6.88,0.02
j2,3.478571,9.857143,0.90000,5.90,0.02
j3,3.459130,9.652174,0.90000,6.00,0.02
j4,3.910455,9.863636,0.90000,6.51,0.02
j5,3.940476,9.047619,0.90000,6.55,0.02


In [167]:
new_df_transposed.to_csv('results3.csv')